# Working with Delta on EMR and Athena

## Add Delta Support to Spark

In [1]:
%%configure -f
{ "conf": 
   {
       "spark.jars.packages": "io.delta:delta-core_2.12:2.1.1",
       "spark.sql.extensions": "io.delta.sql.DeltaSparkSessionExtension",
       "spark.sql.catalog.spark_catalog" : "org.apache.spark.sql.delta.catalog.DeltaCatalog"
   }
}

## Create a SparkSession
We are creating this from scratch because EMR's implicit _SparkSession_ creation is unreliable.

In [2]:
import pyspark # only run after findspark.init()
from pyspark.sql import SparkSession
# May take awhile locally
spark = SparkSession.builder.appName("Export delta to Athena").getOrCreate()
spark

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1668083004963_0001,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Read listings parquet

In [4]:
LISTINGS_INPUT_PARQUET='s3://dt-zoli/listings.parquet/'
LISTINGS_DELTA_LOCATION='s3://athena-delta/delta/listings.delta'
HOSTS_DELTA_LOCATION='s3://athena-delta/delta/hosts.delta'

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
df = spark.read.parquet(LISTINGS_INPUT_PARQUET)
df.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- id: integer (nullable = true)
 |-- listing_url: string (nullable = true)
 |-- name: string (nullable = true)
 |-- room_type: string (nullable = true)
 |-- minimum_nights: integer (nullable = true)
 |-- host_id: integer (nullable = true)
 |-- price: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- updated_at: string (nullable = true)

In [7]:
print(f'This delta file will have {df.rdd.getNumPartitions()} partitions.')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

This delta file will have 3 partitions.

## Write into Delta

In [8]:
df.write.format('delta').save(LISTINGS_DELTA_LOCATION)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
def q(s): return spark.sql(s)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Register listings as a table and execute a few queries 

In [11]:
q(f"DROP TABLE IF EXISTS my_table")
q(f"CREATE TABLE my_table USING DELTA LOCATION '{LISTINGS_DELTA_LOCATION}'")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[]

In [12]:
q('DESCRIBE EXTENDED my_table').show(truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------------------+---------------------------------------------------+-------+
|col_name                    |data_type                                          |comment|
+----------------------------+---------------------------------------------------+-------+
|id                          |int                                                |       |
|listing_url                 |string                                             |       |
|name                        |string                                             |       |
|room_type                   |string                                             |       |
|minimum_nights              |int                                                |       |
|host_id                     |int                                                |       |
|price                       |string                                             |       |
|created_at                  |string                                             |       |

In [13]:
q('SELECT * FROM my_table').show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+--------------------+--------------------+---------------+--------------+-------+-------+--------------------+--------------------+
|   id|         listing_url|                name|      room_type|minimum_nights|host_id|  price|          created_at|          updated_at|
+-----+--------------------+--------------------+---------------+--------------+-------+-------+--------------------+--------------------+
| 3176|https://www.airbn...|Fabulous Flat in ...|Entire home/apt|            62|   3718| $90.00|2009-06-05T21:34:42Z|2009-06-05T21:34:42Z|
| 7071|https://www.airbn...|BrightRoom with s...|   Private room|             1|  17391| $33.00|2009-08-12T12:30:30Z|2009-08-12T12:30:30Z|
| 9991|https://www.airbn...|Geourgeous flat -...|Entire home/apt|             0|  33852|$180.00|2015-07-30T05:08:52Z|2015-07-30T05:08:52Z|
|14325|https://www.airbn...|Apartment in Pren...|Entire home/apt|            95|  55531| $70.00|2010-06-15T19:56:01Z|2010-06-15T19:56:01Z|
|16644|https://www.airbn...

In [15]:
prDF = q("SELECT * FROM my_table WHERE room_type='Private room'")
prDF.write.format('delta').mode('overwrite').save(LISTINGS_DELTA_LOCATION)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
q('DESCRIBE HISTORY my_table').show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-------------------+------+--------+---------+--------------------+----+--------+---------+-----------+--------------+-------------+--------------------+------------+--------------------+
|version|          timestamp|userId|userName|operation| operationParameters| job|notebook|clusterId|readVersion|isolationLevel|isBlindAppend|    operationMetrics|userMetadata|          engineInfo|
+-------+-------------------+------+--------+---------+--------------------+----+--------+---------+-----------+--------------+-------------+--------------------+------------+--------------------+
|      1|2022-11-10 14:30:02|  null|    null|    WRITE|{mode -> Overwrit...|null|    null|     null|          0|  Serializable|        false|{numFiles -> 3, n...|        null|Apache-Spark/3.3....|
|      0|2022-11-10 14:19:57|  null|    null|    WRITE|{mode -> ErrorIfE...|null|    null|     null|       null|  Serializable|         true|{numFiles -> 3, n...|        null|Apache-Spark/3.3....|
+-------+------

In [17]:
hist_df = q('DESCRIBE HISTORY my_table')
hist_df.select('version', 'timestamp', 'operation', 'operationParameters').show(truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-------------------+---------+------------------------------------------+
|version|timestamp          |operation|operationParameters                       |
+-------+-------------------+---------+------------------------------------------+
|1      |2022-11-10 14:30:02|WRITE    |{mode -> Overwrite, partitionBy -> []}    |
|0      |2022-11-10 14:19:57|WRITE    |{mode -> ErrorIfExists, partitionBy -> []}|
+-------+-------------------+---------+------------------------------------------+

In [18]:
q("SELECT * FROM my_table WHERE minimum_nights > 365").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+--------------------+--------------------+------------+--------------+--------+------+--------------------+--------------------+
|      id|         listing_url|                name|   room_type|minimum_nights| host_id| price|          created_at|          updated_at|
+--------+--------------------+--------------------+------------+--------------+--------+------+--------------------+--------------------+
|28677551|https://www.airbn...|Cozy Room and nic...|Private room|          1000|30298984|$20.00|2021-11-13T10:10:47Z|2021-11-13T10:10:47Z|
|17836549|https://www.airbn...|Cozy room in spac...|Private room|          1000| 9194512|$45.00|2017-04-06T12:21:49Z|2017-04-06T12:21:49Z|
|  584757|https://www.airbn...|Nice room trendy ...|Private room|          1000| 2881484|$20.00|2012-08-02T23:58:30Z|2012-08-02T23:58:30Z|
| 4003140|https://www.airbn...|85m² flat near by...|Private room|           500| 5781298|$70.00|2014-09-17T11:46:36Z|2014-09-17T11:46:36Z|
| 6704144|https://www.airbn

In [19]:
q("DELETE FROM my_table WHERE minimum_nights > 365")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[num_affected_rows: bigint]

In [20]:
q("SELECT * FROM my_table WHERE minimum_nights < 1").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+--------------------+--------------------+------------+--------------+---------+-------+--------------------+--------------------+
|      id|         listing_url|                name|   room_type|minimum_nights|  host_id|  price|          created_at|          updated_at|
+--------+--------------------+--------------------+------------+--------------+---------+-------+--------------------+--------------------+
|25049412|https://www.airbn...|Cozy & Bright Roo...|Private room|             0| 11324576| $50.00|2018-09-20T08:36:07Z|2018-09-20T08:36:07Z|
|27655569|https://www.airbn...|Sunny room in the...|Private room|             0|117446866| $18.00|2018-08-07T08:48:22Z|2018-08-07T08:48:22Z|
|27774847|https://www.airbn...|Perfect big room ...|Private room|             0|102845594| $40.00|2018-08-04T10:25:43Z|2018-08-04T10:25:43Z|
|28753356|https://www.airbn...|Nice Flat in Berl...|Private room|             0|216880791| $35.00|2019-05-13T12:29:12Z|2019-05-13T12:29:12Z|
|29501017|htt

In [21]:
q("UPDATE my_table set minimum_nights = 1 WHERE minimum_nights < 1")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[num_affected_rows: bigint]

In [22]:
hist_df = q('DESCRIBE HISTORY my_table')
hist_df.select('version', 'timestamp', 'operation', 'operationParameters').show(truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-------------------+---------+------------------------------------------------------------------------+
|version|timestamp          |operation|operationParameters                                                     |
+-------+-------------------+---------+------------------------------------------------------------------------+
|3      |2022-11-10 14:36:39|UPDATE   |{predicate -> (minimum_nights#2637 < 1)}                                |
|2      |2022-11-10 14:36:14|DELETE   |{predicate -> ["(spark_catalog.default.my_table.minimum_nights > 365)"]}|
|1      |2022-11-10 14:30:02|WRITE    |{mode -> Overwrite, partitionBy -> []}                                  |
|0      |2022-11-10 14:19:57|WRITE    |{mode -> ErrorIfExists, partitionBy -> []}                              |
+-------+-------------------+---------+------------------------------------------------------------------------+

In [23]:
q('SELECT * FROM my_table VERSION AS OF 0').show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+--------------------+--------------------+---------------+--------------+-------+-------+--------------------+--------------------+
|   id|         listing_url|                name|      room_type|minimum_nights|host_id|  price|          created_at|          updated_at|
+-----+--------------------+--------------------+---------------+--------------+-------+-------+--------------------+--------------------+
| 3176|https://www.airbn...|Fabulous Flat in ...|Entire home/apt|            62|   3718| $90.00|2009-06-05T21:34:42Z|2009-06-05T21:34:42Z|
| 7071|https://www.airbn...|BrightRoom with s...|   Private room|             1|  17391| $33.00|2009-08-12T12:30:30Z|2009-08-12T12:30:30Z|
| 9991|https://www.airbn...|Geourgeous flat -...|Entire home/apt|             0|  33852|$180.00|2015-07-30T05:08:52Z|2015-07-30T05:08:52Z|
|14325|https://www.airbn...|Apartment in Pren...|Entire home/apt|            95|  55531| $70.00|2010-06-15T19:56:01Z|2010-06-15T19:56:01Z|
|16644|https://www.airbn...

In [25]:
from delta import DeltaTable

deltaTable = DeltaTable.forPath(spark, path=LISTINGS_DELTA_LOCATION)
deltaTable.generate("symlink_format_manifest")

print(f'Generated Manifest file at {LISTINGS_DELTA_LOCATION}/_symlink_format_manifest')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Generated Manifest file at s3://athena-delta/delta/listings.delta/_symlink_format_manifest

In [26]:
q('OPTIMIZE my_table').show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+
|                path|             metrics|
+--------------------+--------------------+
|s3://athena-delta...|{1, 3, {424100, 4...|
+--------------------+--------------------+

# Load hosts.csv and save as delta

In [4]:
hosts = spark.read.csv('s3://nordquant/athena/airbnb/hosts/hosts.csv', header=True, 
                       schema="id INT, name STRING, is_superhost STRING, created_at STRING, updated_at STRING")
hosts.show()
hosts.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+--------------+------------+-------------------+-------------------+
|   id|          name|is_superhost|         created_at|         updated_at|
+-----+--------------+------------+-------------------+-------------------+
| 1581|       Annette|           f|2014-01-05 16:12:45|2014-01-05 16:12:45|
| 2164|         Lulah|           t|2013-07-31 23:29:31|2013-07-31 23:29:31|
| 2217|           Ion|           t|2017-10-17 05:20:28|2017-10-17 05:20:28|
| 3718|        Britta|           f|2009-06-05 21:34:42|2009-06-05 21:34:42|
|11622|         Maria|           f|2021-10-24 02:42:09|2021-10-24 02:42:09|
|12360|       Michael|           t|2017-08-27 12:08:43|2017-08-27 12:08:43|
|12424|       Mariana|           f|2021-11-08 17:27:31|2021-11-08 17:27:31|
|15115|     Christian|           f|2015-01-03 17:53:53|2015-01-03 17:53:53|
|17391|    BrightRoom|           t|2009-08-12 12:30:30|2009-08-12 12:30:30|
|22901|        Hannes|           f|2013-07-06 11:09:47|2013-07-06 11:09:47|
|32299|     

In [6]:
hosts.write.format("delta").mode("overwrite").option('overwriteSchema', True).save(HOSTS_DELTA_LOCATION)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
from delta import DeltaTable

deltaTable = DeltaTable.forPath(spark, path=HOSTS_DELTA_LOCATION)
deltaTable.generate("symlink_format_manifest")

print(f'Generated Manifest file at {HOSTS_DELTA_LOCATION}/_symlink_format_manifest')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Generated Manifest file at s3://athena-delta/delta/hosts.delta/_symlink_format_manifest